In [0]:
%run /Workspace/Users/ahmadlearn@outlook.com/00_config

In [0]:
from datetime import datetime
from pyspark.sql.functions import col, mean, stddev, when

storage_account_datalake = "salesdatalakedemo"
container_datalake = "salescontainer"

today = datetime.now()
input_folder = (
    f"abfss://{container_datalake}@{storage_account_datalake}.dfs.core.windows.net/raw/{today:%Y/%m/%d}"
)

df = spark.read.option("header", "true").option("inferSchema", "true").csv(input_folder)

stats = df.select(
    mean(col("Sales")).alias("mean_sales"),
    stddev(col("Sales")).alias("std_sales")
).collect()[0]
mean_sales, std_sales = stats["mean_sales"], stats["std_sales"]

df_clean = df.withColumn(
    "Sales_Capped",
    when(
        col("Sales") > mean_sales + 3 * std_sales, mean_sales + 3 * std_sales
    ).when(
        col("Sales") < mean_sales - 3 * std_sales, mean_sales - 3 * std_sales
    ).otherwise(col("Sales"))
)

display(df_clean)

output_folder = (
    f"abfss://{container_datalake}@{storage_account_datalake}.dfs.core.windows.net/stg/{today:%Y/%m/%d}"
)
df_clean.write.mode("overwrite").parquet(output_folder)

In [0]:
%skip
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, when


storage_account_datalake = "salesdatalakedemo"
container_datalake = "salescontainer"

# Get today date folder to verify
today = datetime.now()
input_folder = f"abfss://{container_datalake}@{storage_account_datalake}.dfs.core.windows.net/raw/{today:%Y/%m/%d}"



# Load the dataset
df = spark.read.option("header", "true").option("inferSchema", "true").csv(input_folder)

# Compute mean and std deviation for numeric column (e.g. Sales)
stats = df.select(mean(col("Sales")).alias("mean_sales"), stddev(col("Sales")).alias("std_sales")).collect()[0]
mean_sales, std_sales = stats["mean_sales"], stats["std_sales"]

# Remove or cap outliers (more than 3 std dev from mean)
df_clean = df.withColumn(
    "Sales_Capped",
    col("Sales").when(col("Sales") > mean_sales + 3 * std_sales, mean_sales + 3 * std_sales)
    .when(col("Sales") < mean_sales - 3 * std_sales, mean_sales - 3 * std_sales)
    .otherwise(col("Sales"))
)

display(df_clean)

In [0]:
%skip
# Read raw data from Data Lake Gen2 (reuse same path logic)
from datetime import datetime

storage_account_datalake = "salesdatalakedemo"
container_datalake = "salescontainer"
today = datetime.now()
input_folder = f"abfss://{container_datalake}@{storage_account_datalake}.dfs.core.windows.net/raw/{today:%Y/%m/%d}"

df_raw = spark.read.option("header", "true").csv(input_folder)

# Example cleaning: drop rows with nulls in key columns
df_clean = df_raw.dropna(subset=["Order ID", "Sales"])  # adapt columns as per your data

# Convert Sales to float (example transformation)
from pyspark.sql.functions import col
df_clean = df_clean.withColumn("Sales", col("Sales").cast("float"))

# Show cleaned data sample
df_clean.show(5)

# Register temp view if you want SQL access
df_clean.createOrReplaceTempView("clean_data")
